In [365]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix, accuracy_score

df= pd.read_csv('RELIANCE.NS.csv')
df.head(3)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-02-21,2412.000000,2437.500000,2384.600098,2399.899902,2392.696045,5125968
1,2022-02-22,2352.899902,2395.899902,2347.149902,2389.000000,2381.828857,6227901
2,2022-02-23,2401.100098,2406.550049,2368.000000,2374.050049,2366.923828,3373615


In [366]:
df['Net']=(df['Open']+df['High']+df['Low']+df['Close'])/4
df.drop(['High','Low','Open','Adj Close'],axis=1,inplace=True)
df.head(3)


,Date,Close,Volume,Net
0,2022-02-21,2399.899902,5125968,2408.500000
1,2022-02-22,2389.000000,6227901,2371.237426
2,2022-02-23,2374.050049,3373615,2387.425049


In [367]:
df['MA50']=df['Net'].rolling(50).mean()
df['MA100']=df['Net'].rolling(100).mean()
df

,Date,Close,Volume,Net,MA50,MA100
0,2022-02-21,2399.899902,5125968,2408.500000,NaN,NaN
1,2022-02-22,2389.000000,6227901,2371.237426,NaN,NaN
2,2022-02-23,2374.050049,3373615,2387.425049,NaN,NaN
3,2022-02-24,2255.750000,11466725,2286.012451,NaN,NaN
4,2022-02-25,2283.949951,6724094,2287.062500,NaN,NaN
...,...,...,...,...,...,...
243,2023-02-13,2323.350098,4777674,2331.775024,2498.807748,2505.428502
244,2023-02-14,2378.100098,6609564,2353.399963,2491.929747,2503.876252
245,2023-02-15,2431.949951,15461902,2404.537476,2486.453997,2503.053626
246,2023-02-16,2430.050049,7967558,2442.087524,2481.932498,2502.833126


In [368]:
df=df.sort_values(['Date'],ascending=False)
df.head()

,Date,Close,Volume,Net,MA50,MA100
247,2023-02-17,2440.199951,5450092,2429.575012,2477.452247,2503.218001
246,2023-02-16,2430.050049,7967558,2442.087524,2481.932498,2502.833126
245,2023-02-15,2431.949951,15461902,2404.537476,2486.453997,2503.053626
244,2023-02-14,2378.100098,6609564,2353.399963,2491.929747,2503.876252
243,2023-02-13,2323.350098,4777674,2331.775024,2498.807748,2505.428502


In [369]:
df['Signal']=0
df.loc[df['MA50']>df['MA100'],'Signal']= 1
df.loc[df['MA50']<df['MA100'],'Signal']= -1
df.head(4)


,Date,Close,Volume,Net,MA50,MA100,Signal
247,2023-02-17,2440.199951,5450092,2429.575012,2477.452247,2503.218001,-1
246,2023-02-16,2430.050049,7967558,2442.087524,2481.932498,2502.833126,-1
245,2023-02-15,2431.949951,15461902,2404.537476,2486.453997,2503.053626,-1
244,2023-02-14,2378.100098,6609564,2353.399963,2491.929747,2503.876252,-1


In [370]:
df.loc[df['Signal']==1,'Signal'].count()

45

In [371]:
df['MA50'] = df['MA50'].fillna(0)
df['MA100'] = df['MA100'].fillna(0)

In [372]:
X=df.drop(['Signal','Date'],axis=1)
y= df['Signal']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
model= LinearRegression()
X.head(3)

,Close,Volume,Net,MA50,MA100
247,2440.199951,5450092,2429.575012,2477.452247,2503.218001
246,2430.050049,7967558,2442.087524,2481.932498,2502.833126
245,2431.949951,15461902,2404.537476,2486.453997,2503.053626


In [374]:
y_test.head()

0      0
79     0
171   -1
97     0
102   -1
Name: Signal, dtype: int64

In [375]:
model.fit(X_train,y_train)

LinearRegression()

In [376]:
y_pred= model.predict(X_test)

In [377]:
confusion = confusion_matrix(y_test, y_pred.round())
confusion

array([[ 3, 19,  0],
       [ 0, 22,  0],
       [ 0,  6,  0]], dtype=int64)

In [378]:
x= pd.DataFrame({'Close':[2880],'Volume':[545009],'Net':[2871],'MA50':[2490],'MA100':[2492]})
x

,Close,Volume,Net,MA50,MA100
0,2880,545009,2871,2490,2492


In [379]:
accuracy = accuracy_score(y_test, y_pred.round())
accuracy

0.5

In [380]:
y= model.predict(x)

In [381]:
df_pred = pd.DataFrame(y_pred, columns=['Signal'])
df_pred.head()

,Signal
0,0.117256
1,-0.031606
2,-0.342692
3,0.221308
4,-0.337607


In [388]:
f = pd.concat([y_test,df_pred], axis=1)
f


,Signal,Signal
0,0.0,0.117256
79,0.0,NaN
171,-1.0,NaN
97,0.0,NaN
102,-1.0,NaN
...,...,...
42,NaN,-0.396650
43,NaN,0.141903
47,NaN,-0.432400
48,NaN,-0.400148
